In [3]:
from sklearn.datasets import make_classification
import torch
import torch.nn as nn

In [7]:
#step 1:create a synthetc classification dataset using sklearn
X, y = make_classification(
    n_samples=10, # Reduced number of samples for simplicity
    n_features=2, # Reduced number of features for simplicity
    n_informative=2, # All features are informative
    n_redundant=0,# Reduced number of redundant features for simplicity
    n_classes=2, # Binary classification
    random_state=42) #reproducibility

In [17]:
# Step 2: Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [18]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [19]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [20]:
from torch.utils.data import Dataset, DataLoader # load the Dataset and DataLoader classes from utils.data

In [21]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        self.features =features
        self.labels = labels

    def __len__(self):
        self.features.shape[0] # return the number of samples
        
    def __getitem__(self, index):
        return self.features[index], self.labels[index] # return the feature and label at the given index

In [22]:
dataset = CustomDataset(X,y)

In [26]:
dataset[3]

(tensor([-0.7206, -0.9606]), tensor(0))

In [28]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)

# Correct way
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Example: iterate through batches
for batch_X, batch_y in loader:
    print(batch_X, batch_y)


tensor([[-0.5872, -1.9717],
        [ 1.0683, -0.9701]]) tensor([0, 1])
tensor([[-0.9382, -0.5430],
        [-0.7206, -0.9606]]) tensor([1, 0])
tensor([[ 1.7774,  1.5116],
        [-1.9629, -0.9923]]) tensor([1, 0])
tensor([[ 1.7273, -1.1858],
        [-1.1402, -0.8388]]) tensor([1, 0])
tensor([[-2.8954,  1.9769],
        [ 1.8997,  0.8344]]) tensor([0, 1])


In [ ]:
# create a DataLoader with a batch size of 2 and shuffling enabled
DataLoader = DataLoader(dataset, batch_size=2, shuffle=True) 

In [30]:
for batch_features, batch_labels in DataLoader:
    print(batch_features)  # Print the features and labels in each batch
    print(batch_labels)
    print("-"*50)  # Separator for clarity

tensor([[-0.9382, -0.5430],
        [ 1.8997,  0.8344]])
tensor([1, 1])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-2.8954,  1.9769]])
tensor([1, 0])
--------------------------------------------------
tensor([[ 1.7774,  1.5116],
        [-0.5872, -1.9717]])
tensor([1, 0])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [-1.1402, -0.8388]])
tensor([0, 0])
--------------------------------------------------


In [31]:
# above the functionality to apply on the real dataset

In [32]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [33]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [34]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [36]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [38]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [39]:
from torch.utils.data import Dataset, DataLoader # load the Dataset and DataLoader classes from utils.data
class CustomDataset(Dataset):


    def __init__(self, features, labels):
        self.features =features
        self.labels = labels

    def __len__(self):
        self.features.shape[0] # return the number of samples
        
    def __getitem__(self, index):
        return self.features[index], self.labels[index] # return the feature and label at the given index

In [49]:
# Convert to tensors (if not already)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

In [51]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)


In [52]:
train_dataset[0]

(tensor([-0.7789, -0.1258, -0.7800, -0.7448,  0.6000, -0.3705, -0.5743, -0.5849,
          0.4009, -0.1141, -0.5927, -0.3802, -0.6916, -0.5111,  0.5080, -0.5073,
         -0.3702, -0.1159,  0.4561, -0.9422, -0.7718, -0.4048, -0.8242, -0.7016,
          0.6994, -0.6153, -0.5223, -0.4303,  0.2549, -0.9000]),
 tensor(0))

In [56]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset  = TensorDataset(X_test, y_test)


In [57]:
# create a DataLoader with a batch size of 2 and shuffling enabled
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [58]:
train_loader

In [61]:
import torch.nn as nn
class MySimpleNN(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid() 


  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

In [62]:
learning_rate = 0.1
epochs = 25

In [60]:
loss_function = nn.BCELoss()

In [64]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


  # loss calculate
  # loss = loss_function(y_pred, y_train_tensor.view(-1, 1))  # Ensure y_train_tensor is the correct shape for BCELoss
loss_function = nn.BCELoss()


  # #clear gradients
  # optimizer.zero_grad()

  # # backward pass
  # loss.backward()

  # # parameters update
  # optimizer.step()
  # # zero gradients
  # model.linear.weight.grad.zero_()
  # model.linear.bias.grad.zero_()

  # # print loss in each epoch
  # print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

In [66]:
#training pipeline
for epoch in range(epochs):
  for batch_features, batch_labels in train_loader:
    # forward pass
    y_pred = model(batch_features)      
    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1, 1).float())  # Ensure batch_labels is the correct shape for BCELoss

    # clear gradients
    optimizer.zero_grad()       
    # backward pass
    loss.backward()
    # parameters update
    optimizer.step()        
    # zero gradients
    model.linear.weight.grad.zero_()
    model.linear.bias.grad.zero_()
    # print loss in each epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')
# Example: iterate through batches
for batch_features, batch_labels in train_loader:
    print(batch_features)  # Print the features and labels in each batch
    print(batch_labels)
    print("-"*50)   
# Example: iterate through batches
for batch_features, batch_labels in test_loader:
    print(batch_features)  # Print the features and labels in each batch
    print(batch_labels)
    print("-"*50)   


Epoch: 1, Loss: 0.4685640335083008
Epoch: 1, Loss: 0.44156521558761597
Epoch: 1, Loss: 0.36181020736694336
Epoch: 1, Loss: 0.2856224477291107
Epoch: 1, Loss: 0.3019724488258362
Epoch: 1, Loss: 0.38349950313568115
Epoch: 1, Loss: 0.304193913936615
Epoch: 1, Loss: 0.2595657706260681
Epoch: 1, Loss: 0.2202347218990326
Epoch: 1, Loss: 0.27859944105148315
Epoch: 1, Loss: 0.2136702984571457
Epoch: 1, Loss: 0.22826515138149261
Epoch: 1, Loss: 0.17071549594402313
Epoch: 1, Loss: 0.19875620305538177
Epoch: 1, Loss: 0.17885783314704895
Epoch: 2, Loss: 0.20947767794132233
Epoch: 2, Loss: 0.2026069462299347
Epoch: 2, Loss: 0.11897829920053482
Epoch: 2, Loss: 0.22972168028354645
Epoch: 2, Loss: 0.14249178767204285
Epoch: 2, Loss: 0.26141858100891113
Epoch: 2, Loss: 0.11589393019676208
Epoch: 2, Loss: 0.10250361263751984
Epoch: 2, Loss: 0.1422826498746872
Epoch: 2, Loss: 0.24601295590400696
Epoch: 2, Loss: 0.1299126148223877
Epoch: 2, Loss: 0.21295660734176636
Epoch: 2, Loss: 0.1880219280719757
Epoc

In [67]:
#model evaluation using test dataset
model.eval()    
accuracy_list = []
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        y_pred = model(batch_features)
        y_pred= (y_pred > 0.5).float()
    # Calculate accuracy
    batch_accuracy = (y_pred == batch_labels.view(-1, 1).float()).float().mean()
    accuracy_list.append(batch_accuracy)

#calculate average accuracy
average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Average Test Accuracy: {average_accuracy:.4f}')

Average Test Accuracy: 1.0000
